In [11]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [29]:
from collections import defaultdict

## Download the data

In [2]:
# File available in the code repo

# # Wget the works of Jane Austen
# !wget http://www.gutenberg.org/files/31100/31100.txt

--2020-10-06 21:03:45--  http://www.gutenberg.org/files/31100/31100.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4454075 (4.2M) [text/plain]
Saving to: ‘31100.txt’

31100.txt           100%[===================>]   4.25M   566KB/s    in 12s     

2020-10-06 21:03:59 (358 KB/s) - ‘31100.txt’ saved [4454075/4454075]



In [73]:
np.random.seed(42)

In [26]:
with open("31100.txt", "r") as f:
    text = f.readlines()

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/shruti/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)

Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x7f07c5fda830>
Traceback (most recent call last):
  File "/home/shruti/.local/lib/python3.7/site-packages/nltk/data.py", line 1161, in __del__
    if not self.closed:
  File "/home/shruti/.local/lib/python3.7/site-packages/nltk/data.py", line 1175, in closed
    return self.stream.closed
AttributeError: 'SeekableUnicodeStreamReader' object has no attribute 'stream'


In [31]:
len(vectorizer.get_feature_names())

15354

In [48]:
counts = X.toarray().sum(axis=0)

In [49]:
word_dict = dict(zip(vectorizer.get_feature_names(),counts))

In [51]:
sorted_word_dict = sorted(word_dict.items(), key=lambda x: x[1], reverse=True)

In [54]:
sorted_word_dict[0:10]

[('the', 28621),
 ('to', 26199),
 ('and', 24238),
 ('of', 23163),
 ('her', 13929),
 ('in', 12278),
 ('was', 11762),
 ('it', 10856),
 ('she', 10619),
 ('not', 9085)]

In [57]:
# Retain top 100 most frequently occurring words after removing the stopwords

vocab_dict = defaultdict(int)

counter = 0
for tup in sorted_word_dict:
    if counter == 100:
        break
    if not tup[0] in stopwords.words():
        vocab_dict[tup[0]] = tup[1]
        counter += 1

In [62]:
vocab_to_id_map = defaultdict(int)
id_to_vocab_map = defaultdict(int)

for idx, i in enumerate(list(vocab_dict.keys())):
    vocab_to_id_map[i] = idx
    id_to_vocab_map[idx] = i

In [59]:
len(vocab_dict), list(vocab_dict.keys())[0:10]

(100,
 ['could',
  'would',
  'mr',
  'mrs',
  'must',
  'said',
  'much',
  'miss',
  'well',
  'every'])

In [ ]:
k = 20
v = 100
d = 2000

alpha = 0.01
beta = 0.01

### Initial topics

In [63]:
topic_ids = list(np.random.randint(0, 101, 20))
print("Topic Ids: ", topic_ids)

for i in topic_ids:
    print(id_to_vocab_map[i], end=", ")

Topic Ids:  [26, 99, 84, 34, 50, 47, 94, 65, 74, 35, 18, 70, 82, 67, 18, 57, 11, 32, 90, 86]
lady, yes, love, quite, elizabeth, sure, really, friend, crawford, made, though, jane, yet, enough, though, many, never, sir, letter, brother, 

In [85]:
topic_to_id_map = defaultdict(int)
id_to_topic_map = defaultdict(int)

for idx, i in enumerate(topic_ids):
    topic_to_id_map[i] = idx
    id_to_topic_map[idx] = i

In [70]:
def generate_distributions():
    fs = np.random.uniform(0, 1, 20)
    tup = []
    for i in fs:
        tup.append(int(i*100))
    topic_distribution = np.random.dirichlet(tup, 2000)

    fs = np.random.uniform(0, 1, 20)
    tup = []
    for i in fs:
        tup.append(int(i*100))
    word_distribution = np.random.dirichlet(tup, 100)
    
    return topic_distribution, word_distribution

In [71]:
topic_distribution, word_distribution = generate_distributions()

In [72]:
topic_distribution.shape, word_distribution.shape

((2000, 20), (100, 20))

In [116]:
def generate_doc():
    random_idx = np.random.randint(0, 2000)
    cumulative_topic_probs_for_idx = defaultdict(float)

    for idx, i in enumerate(topic_distribution[random_idx]):
        if idx > 0:
            cumulative_topic_probs_for_idx[idx] = cumulative_topic_probs_for_idx[idx-1] + i
        else:
            cumulative_topic_probs_for_idx[idx] = i

    # print(topic_distribution[random_idx])
    # print(cumulative_topic_probs_for_idx)

    # Selecting a topic for the document
    select_topic_prob = np.random.uniform(0, 1)

    selected_topic_id_for_doc_gen = None
    for i in range(0, 19):
        if select_topic_prob > cumulative_topic_probs_for_idx[i] and select_topic_prob < cumulative_topic_probs_for_idx[i+1]:
            selected_topic_id_for_doc_gen = i

    if not selected_topic_id_for_doc_gen:
        selected_topic_id_for_doc_gen = 19


    # Calculate the cumulative prob for the words
    cumulative_word_probs_for_idx = defaultdict(float)
    for idx, i in enumerate(word_distribution[selected_topic_id_for_doc_gen]):
        if idx > 0:
            cumulative_word_probs_for_idx[idx] = cumulative_word_probs_for_idx[idx-1] + i
        else:
            cumulative_word_probs_for_idx[idx] = i

    document_words = []
    # Generate 100 words for the document
    for w in range(0, 100):
        # Selecting a word prob
        select_word_prob = np.random.uniform(0, 1)

        selected_word_id_for_topic = None
        for i in range(0, 99):
            if select_word_prob > cumulative_word_probs_for_idx[i] and select_word_prob < cumulative_word_probs_for_idx[i+1]:
                selected_word_id_for_topic = i

        if not selected_word_id_for_topic:
            selected_word_id_for_topic = 99
        document_words.append(selected_word_id_for_topic)
    
    return random_idx, document_words

In [117]:
did, doc_words = generate_doc()
print(did, doc_words)

1252 [1, 2, 15, 13, 8, 6, 15, 8, 15, 8, 7, 8, 4, 13, 8, 2, 16, 7, 9, 16, 11, 1, 16, 11, 6, 1, 14, 11, 9, 16, 14, 1, 4, 3, 13, 99, 10, 13, 15, 6, 15, 1, 15, 1, 8, 14, 1, 2, 13, 13, 11, 13, 9, 3, 6, 2, 16, 10, 8, 8, 16, 1, 10, 8, 11, 99, 15, 16, 10, 13, 16, 13, 1, 10, 11, 8, 13, 16, 16, 8, 1, 17, 15, 1, 16, 15, 8, 10, 8, 99, 6, 14, 99, 6, 8, 9, 16, 6, 6, 13]


In [118]:
document_text = []
for i in doc_words:
    document_text.append(id_to_vocab_map[i])

print(document_text)

['would', 'mr', 'time', 'good', 'well', 'much', 'time', 'well', 'time', 'well', 'miss', 'well', 'must', 'good', 'well', 'mr', 'little', 'miss', 'every', 'little', 'never', 'would', 'little', 'never', 'much', 'would', 'might', 'never', 'every', 'little', 'might', 'would', 'must', 'mrs', 'good', 'yes', 'think', 'good', 'time', 'much', 'time', 'would', 'time', 'would', 'well', 'might', 'would', 'mr', 'good', 'good', 'never', 'good', 'every', 'mrs', 'much', 'mr', 'little', 'think', 'well', 'well', 'little', 'would', 'think', 'well', 'never', 'yes', 'time', 'little', 'think', 'good', 'little', 'good', 'would', 'think', 'never', 'well', 'good', 'little', 'little', 'well', 'would', 'nothing', 'time', 'would', 'little', 'time', 'well', 'think', 'well', 'yes', 'much', 'might', 'yes', 'much', 'well', 'every', 'little', 'much', 'much', 'good']


In [120]:
prob_of_doc = 0
log_wi_given_d = []

for wi in doc_words:
    wi_given_d = 0
    for ti in range(0, 20):
        wi_given_d += word_distribution[wi][ti]*topic_distribution[did][ti]
    log_wi_given_d.append(np.log(wi_given_d))
print("Log likelihood of the generated text: ", np.sum(log_wi_given_d))

Log likelihood of the generated text:  -307.53754115389484


In [124]:
did, doc_words = generate_doc()
print(did, doc_words)

prob_of_doc = 0
log_wi_given_d = []

for wi in doc_words:
    wi_given_d = 0
    for ti in range(0, 20):
        wi_given_d += word_distribution[wi][ti]*topic_distribution[did][ti]
    log_wi_given_d.append(np.log(wi_given_d))
print("Log likelihood of the generated text: ", np.sum(log_wi_given_d))

1711 [1, 15, 8, 8, 11, 15, 6, 13, 10, 4, 15, 14, 13, 15, 15, 13, 15, 13, 13, 12, 13, 2, 16, 15, 99, 15, 1, 6, 13, 2, 15, 15, 9, 99, 4, 11, 17, 12, 4, 12, 9, 14, 1, 13, 9, 17, 6, 12, 16, 1, 16, 13, 99, 5, 13, 99, 11, 6, 12, 99, 15, 17, 17, 13, 1, 13, 99, 99, 6, 8, 13, 13, 8, 4, 17, 8, 8, 2, 14, 13, 2, 1, 13, 13, 4, 16, 1, 8, 16, 8, 12, 8, 4, 17, 8, 16, 3, 2, 99, 13]
Log likelihood of the generated text:  -304.30582649510853
